# Get Data

In [1]:
import pandas as pd
from nba_api.stats.endpoints import leagueleaders

In [2]:
# Sample 
leaders = leagueleaders.LeagueLeaders(season = '1990-91', per_mode48= 'PerGame').get_data_frames()[0]

In [3]:
leaders.head()

,PLAYER_ID,RANK,PLAYER,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF
0,893,1,Michael Jordan,CHI,82,37.0,12.1,22.4,0.539,0.4,...,0.851,1.4,4.6,6.0,5.5,2.7,1.0,2.5,31.5,32.7
1,252,2,Karl Malone,UTH,82,40.3,10.3,19.6,0.527,0.0,...,0.770,2.9,8.9,11.8,3.3,1.1,1.0,3.0,29.0,31.4
2,77264,3,Bernard King,WAS,64,37.5,11.1,23.6,0.472,0.1,...,0.790,1.8,3.2,5.0,4.6,0.9,0.3,4.0,28.4,21.0
3,787,4,Charles Barkley,PHL,67,37.3,9.9,17.4,0.570,0.7,...,0.722,3.9,6.3,10.1,4.2,1.6,0.5,3.1,27.6,30.8
4,121,5,Patrick Ewing,NYK,81,38.3,10.4,20.3,0.514,0.0,...,0.745,2.4,8.8,11.2,3.0,1.0,3.2,3.6,26.6,29.5


In [4]:
def get_season_code(start_year):
    # Properly format numeric year to NBA's season format
    season_start = str(start_year)
    season_end = str(int(season_start) + 1)[-2:]
    season = season_start + "-" + str(season_end)
    return season

In [5]:
retired_nicknames = {'SAN':'Clippers', 'KCK':'Kings', 'GOS':'Warriors', 'NJN':'Nets',
                    'PHL':'76ers', 'UTH':'Jazz', 'SEA':'SuperSonics',
                    'CHH': 'Hornets', 'SDC': 'Clippers', 'VAN': 'Grizzlies',
                    'NOH': 'Hornets', 'NOK': 'Pelicans'}
relocated= {'SAN':'LAC', 'KCK':'SAC', 'GOS':'GSW', 'NJN':'BKN',
           'PHL':'PHI', 'UTH':'UTA', 'UTJ':'UTA', 'CHH':'CHA', 'SDC':'LAC',
           'VAN':'MEM', 'NOH':'CHA', 'NOK':'NOP'}
old_names = {'Bullets':'WAS', 'SuperSonics':'SEA'}

In [6]:
def replace_relocated(df1):
    df = df1.copy()
    for index, row in df.iterrows():
        team = row['TEAM']
        if team in relocated:
            new = relocated[team]
            df['TEAM'][index] = new
        else:
            pass
    return df

In [55]:
def clean_year(df1, season):
    df = df1.copy()
    # Add column indicating season
    df['SEASON'] = season
    df = replace_relocated(df)
    df['TS%'] = df['PTS'] / (2*(df['FGA'] + .44*df['FTA']))
    df = df[['PLAYER', 'TEAM', 'SEASON', 'GP', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 
        'REB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'EFF']].copy()
    return df

def get_year_data(start_year):
    year = start_year
    season = get_season_code(year)
    #print(season)
    leaders = leagueleaders.LeagueLeaders(season = season, per_mode48= 'PerGame')
    leaders = leaders.get_data_frames()[0]
    leaders = clean_year(leaders, season)
    return leaders

def get_half_decade_data(start_year):
    lst = []
    for x in range(0, 5):
        year = start_year + x
        leaders = get_year_data(year)
        lst.append(leaders)
    df = pd.concat(lst)
    # Add percentiles for FTA and 3PA
    df['FTA PCTL'] =  df['FTA'].rank(pct=True).round(1)
    df['FG3A PCTL'] = df['FG3A'].rank(pct=True).round(1)
    return df    

## Get data in 5 year groups

In [56]:
years80_85 = get_half_decade_data(1980)
years85_90 = get_half_decade_data(1985)

<ipython-input-6-e4618792ede5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TEAM'][index] = new


In [57]:
years90_95 = get_half_decade_data(1990)
years95_00 = get_half_decade_data(1995)

<ipython-input-6-e4618792ede5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TEAM'][index] = new


In [58]:
years00_05 = get_half_decade_data(2000)
years05_10 = get_half_decade_data(2005)

<ipython-input-6-e4618792ede5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TEAM'][index] = new


In [59]:
years10_15 = get_half_decade_data(2010)
years15_20 = get_half_decade_data(2015)

<ipython-input-6-e4618792ede5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TEAM'][index] = new


In [60]:
all_data = pd.concat([years80_85, years85_90, years90_95, years95_00, years00_05, years05_10,
                      years10_15, years15_20])

In [14]:
all_data.columns

Index(['PLAYER', 'TEAM', 'SEASON', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'TS%', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PTS', 'EFF', 'FTA PCTL', 'FG3A PCTL'],
      dtype='object')

In [100]:
all_data = all_data.fillna(0.0)
all_data['TS_PCT'] = (all_data['PTS'] / (2*(all_data['FGA'] + .44*all_data['FTA']))).round(3)
all_data.head()

,PLAYER,TEAM,SEASON,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,REB,AST,STL,BLK,TOV,PTS,EFF,FTA PCTL,FG3A PCTL,TS_PCT
0,Adrian Dantley,UTA,1980-81,80,42.7,11.4,20.3,0.559,0.0,0.1,...,6.4,4.0,1.4,0.2,3.5,30.7,28.2,1.0,0.5,0.624
1,Moses Malone,HOU,1980-81,80,40.6,10.1,19.3,0.522,0.0,0.0,...,14.8,1.8,1.0,1.9,3.9,27.8,31.7,1.0,0.2,0.585
2,George Gervin,LAC,1980-81,82,33.7,10.4,21.1,0.492,0.1,0.4,...,5.1,3.2,1.1,0.7,3.1,27.1,22.1,1.0,0.8,0.554
3,Kareem Abdul-Jabbar,LAL,1980-81,80,37.2,10.5,18.2,0.574,0.0,0.0,...,10.3,3.4,0.7,2.9,3.1,26.2,31.0,1.0,0.2,0.617
4,David Thompson,DEN,1980-81,77,34.0,9.5,18.8,0.506,0.1,0.5,...,3.7,3.0,0.7,0.8,3.2,25.5,19.5,1.0,0.8,0.571


In [101]:
all_data.to_csv('All_Players.csv')

# Manipulate Data

In [63]:
averages_df = all_data.groupby(by=['SEASON']).mean().round(3)

### Examine Historic trends

In [64]:
averages_df

,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,REB,AST,STL,BLK,TOV,PTS,EFF,FTA PCTL,FG3A PCTL
SEASON,,,,,,,,,,,,,,,,,,,,
1980-81,78.963,27.144,5.011,10.167,0.489,0.048,0.217,0.138,2.554,3.372,0.750,4.919,2.837,1.011,0.623,2.060,12.639,13.984,0.519,0.478
1981-82,78.347,26.443,4.831,9.733,0.491,0.052,0.219,0.165,2.389,3.183,0.739,4.842,2.843,0.942,0.596,1.925,12.104,13.700,0.492,0.479
1982-83,78.375,26.568,4.960,10.082,0.488,0.053,0.232,0.144,2.384,3.188,0.735,5.044,2.810,0.973,0.663,2.073,12.359,13.841,0.485,0.502
1983-84,78.556,26.414,4.893,9.856,0.493,0.057,0.248,0.152,2.573,3.358,0.756,4.749,2.819,0.931,0.596,1.920,12.417,13.832,0.508,0.514
1984-85,78.071,26.981,5.046,10.190,0.494,0.107,0.362,0.165,2.604,3.382,0.762,4.793,3.073,0.969,0.586,1.978,12.805,14.322,0.503,0.547
1985-86,78.120,26.517,4.874,9.879,0.492,0.094,0.346,0.155,2.557,3.351,0.753,4.797,2.923,0.983,0.590,1.914,12.401,13.971,0.501,0.442
1986-87,78.327,27.584,5.008,10.269,0.485,0.175,0.555,0.170,2.750,3.571,0.759,5.122,3.031,0.977,0.661,1.881,12.940,14.781,0.527,0.472
1987-88,78.110,26.257,4.697,9.638,0.484,0.163,0.523,0.173,2.515,3.269,0.764,4.916,2.863,0.929,0.597,1.770,12.079,13.918,0.485,0.453
1988-89,78.167,27.358,4.961,10.272,0.479,0.252,0.781,0.189,2.619,3.372,0.763,5.079,2.957,1.023,0.635,1.905,12.800,14.513,0.501,0.538


## Scale percentage of threes and free throws to percentile of era

In [65]:
yr80_FG3A = years80_85[['FG3A', 'FG3A PCTL']].groupby(by=['FG3A PCTL']).mean()
yr80_FG3A = years85_90[['FG3A', 'FG3A PCTL']].groupby(by=['FG3A PCTL']).mean()

yr90_FG3A = years90_95[['FG3A', 'FG3A PCTL']].groupby(by=['FG3A PCTL']).mean()
yr95_FG3A = years95_00[['FG3A', 'FG3A PCTL']].groupby(by=['FG3A PCTL']).mean()

yr00_FG3A = years00_05[['FG3A', 'FG3A PCTL']].groupby(by=['FG3A PCTL']).mean()
yr05_FG3A = years05_10[['FG3A', 'FG3A PCTL']].groupby(by=['FG3A PCTL']).mean()

yr10_FG3A = years10_15[['FG3A', 'FG3A PCTL']].groupby(by=['FG3A PCTL']).mean()
yr15_FG3A = years15_20[['FG3A', 'FG3A PCTL']].groupby(by=['FG3A PCTL']).mean()



combined_FG3A = pd.concat([yr80_FG3A, yr80_FG3A, yr90_FG3A, yr95_FG3A,
                          yr00_FG3A, yr05_FG3A, yr10_FG3A, yr15_FG3A], axis=1).round(2)
combined_FG3A.columns = ['80s', '85s', '90s', '95s', '00s', '05s', '10s', '15s']

In [66]:
combined_FG3A

,80s,85s,90s,95s,00s,05s,10s,15s
FG3A PCTL,,,,,,,,
0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
0.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.18
0.2,NaN,NaN,NaN,NaN,NaN,0.10,0.10,0.97
0.3,NaN,NaN,0.10,0.13,0.16,0.36,0.37,1.74
0.4,0.10,0.10,0.25,0.47,0.70,1.10,1.29,2.36
0.5,0.20,0.20,0.44,1.11,1.44,1.92,2.12,2.87
0.6,0.34,0.34,0.77,1.87,2.07,2.59,2.74,3.46
0.7,0.58,0.58,1.38,2.62,2.71,3.29,3.34,4.12
0.8,1.03,1.03,2.19,3.36,3.41,3.94,4.02,4.87


In [67]:
yr80_FTA = years80_85[['FTA', 'FTA PCTL']].groupby(by=['FTA PCTL']).mean()
yr80_FTA = years85_90[['FTA', 'FTA PCTL']].groupby(by=['FTA PCTL']).mean()

yr90_FTA = years90_95[['FTA', 'FTA PCTL']].groupby(by=['FTA PCTL']).mean()
yr95_FTA = years95_00[['FTA', 'FTA PCTL']].groupby(by=['FTA PCTL']).mean()

yr00_FTA = years00_05[['FTA', 'FTA PCTL']].groupby(by=['FTA PCTL']).mean()
yr05_FTA = years05_10[['FTA', 'FTA PCTL']].groupby(by=['FTA PCTL']).mean()

yr10_FTA = years10_15[['FTA', 'FTA PCTL']].groupby(by=['FTA PCTL']).mean()
yr15_FTA = years15_20[['FTA', 'FTA PCTL']].groupby(by=['FTA PCTL']).mean()



combined_FTA = pd.concat([yr80_FTA, yr80_FTA, yr90_FTA, yr95_FTA,
                          yr00_FTA, yr05_FTA, yr10_FTA, yr15_FTA], axis=1).round(2)
combined_FTA.columns = ['80s', '85s', '90s', '95s', '00s', '05s', '10s', '15s']

In [68]:
combined_FTA

,80s,85s,90s,95s,00s,05s,10s,15s
FTA PCTL,,,,,,,,
0.0,0.68,0.68,0.61,0.63,0.63,0.54,0.47,0.42
0.1,1.12,1.12,1.19,1.13,1.07,0.97,0.88,0.71
0.2,1.59,1.59,1.60,1.51,1.41,1.34,1.20,0.99
0.3,2.05,2.05,1.95,1.86,1.74,1.74,1.51,1.30
0.4,2.48,2.48,2.33,2.24,2.14,2.16,1.80,1.59
0.5,3.01,3.01,2.75,2.65,2.54,2.55,2.20,1.91
0.6,3.53,3.53,3.26,3.08,3.00,3.01,2.65,2.29
0.7,4.14,4.14,3.81,3.63,3.60,3.55,3.15,2.78
0.8,4.88,4.88,4.46,4.43,4.36,4.43,3.83,3.47


In [119]:
# Adjust FTA to 80s and FG3A to 15s
def adjust_FTA(row):
    percentile = row['FTA PCTL']
    adjusted_FTA = combined_FTA.loc[percentile][0]
    actual_FTA = row['FTA']
    if actual_FTA > adjusted_FTA:
        FTA = actual_FTA
    else:
        FTA = adjusted_FTA
    return FTA

def adjust_FG3A(row):
    percentile = row['FG3A PCTL']
    adjusted_FG3A = combined_FG3A.loc[percentile][7]
    actual_FG3A = row['FG3A']
    if actual_FG3A > adjusted_FG3A:
        FG3A = actual_FG3A
    else:
        FG3A = adjusted_FG3A
    return FG3A


In [131]:
def adjust_stats(df):
    # Pull players from season
    players = df.copy()
    # Make adjustments
    players['FTA'] = players.apply(adjust_FTA, axis=1)
    players['FG3A'] = players.apply(adjust_FG3A, axis=1)
    players['FGM'] = (players['FGA']*players['FG_PCT'])
    players['FTM'] = (players['FTA']*players['FT_PCT'])
    players['FG3M'] = players['FG3A'] * players['FG3_PCT']
    players['PTS'] = (2*(players['FGM']-players['FG3M']) + 3*players['FG3M'] + players['FTM']).round(2)
    players['TS_PCT'] = players['PTS'] / (2*(players['FGA'] + .44*players['FTA']))
    return players.round(3)

In [132]:
all_adjusted = adjust_stats(all_data).reset_index(drop=True)

In [134]:
all_adjusted[['PLAYER', 'TEAM', 'SEASON', 'FGM', 'FG3M', 'FG3_PCT', 'FG3A PCTL', 'FTM', 'FTA PCTL',
                      'TS_PCT', 'AST', 'REB', 'STL', 'BLK', 'TOV', 'PTS', 'EFF']].sort_values(by=['PTS'], ascending=False).head(10)

,PLAYER,TEAM,SEASON,FGM,FG3M,FG3_PCT,FG3A PCTL,FTM,FTA PCTL,TS_PCT,AST,REB,STL,BLK,TOV,PTS,EFF
1032,Michael Jordan,CHI,1986-87,13.400,0.886,0.182,0.8,10.198,1.0,0.573,4.6,5.2,2.9,1.5,3.3,37.88,31.9
7287,James Harden,HOU,2018-19,10.829,4.858,0.368,1.0,9.669,1.0,0.617,7.5,6.6,2.0,0.7,5.0,36.18,33.1
4486,Kobe Bryant,LAL,2005-06,12.240,2.845,0.347,1.0,8.670,1.0,0.568,4.5,5.3,1.8,0.4,3.1,36.00,27.8
1535,Michael Jordan,CHI,1989-90,12.624,3.083,0.376,1.0,7.361,1.0,0.641,6.3,6.9,2.8,0.7,3.0,35.69,34.6
1191,Michael Jordan,CHI,1987-88,13.054,0.544,0.132,0.7,8.830,1.0,0.611,5.9,5.5,3.2,1.6,3.1,35.48,35.0
2111,Michael Jordan,CHI,1992-93,12.721,2.116,0.352,0.9,7.265,1.0,0.590,5.5,6.7,2.8,0.8,2.7,34.82,31.6
7545,James Harden,HOU,2019-20,9.901,4.402,0.355,1.0,10.207,1.0,0.626,7.5,6.6,1.8,0.9,4.5,34.41,32.6
162,George Gervin,LAC,1981-82,12.600,1.354,0.278,0.8,7.500,1.0,0.587,2.4,5.0,1.0,0.6,2.7,34.05,24.8
1355,Michael Jordan,CHI,1988-89,11.944,1.344,0.276,0.8,8.330,1.0,0.633,8.0,8.0,2.9,0.8,3.6,33.56,37.0
4487,Allen Iverson,PHI,2005-06,11.309,1.331,0.323,0.7,9.361,1.0,0.549,7.4,3.2,1.9,0.1,3.4,33.31,26.1


In [135]:
all_data[['PLAYER', 'TEAM', 'SEASON', 'FGM', 'FG3M', 'FG3_PCT', 'FG3A PCTL', 'FTM', 'FTA PCTL',
                'TS_PCT', 'AST', 'REB', 'STL', 'BLK', 'TOV', 'PTS', 'EFF']].reset_index(drop=True).sort_values(by=['PTS'], ascending=False).head(10)

,PLAYER,TEAM,SEASON,FGM,FG3M,FG3_PCT,FG3A PCTL,FTM,FTA PCTL,TS_PCT,AST,REB,STL,BLK,TOV,PTS,EFF
1032,Michael Jordan,CHI,1986-87,13.4,0.1,0.182,0.8,10.2,1.0,0.562,4.6,5.2,2.9,1.5,3.3,37.1,31.9
7287,James Harden,HOU,2018-19,10.8,4.8,0.368,1.0,9.7,1.0,0.615,7.5,6.6,2.0,0.7,5.0,36.1,33.1
4486,Kobe Bryant,LAL,2005-06,12.2,2.3,0.347,1.0,8.7,1.0,0.559,4.5,5.3,1.8,0.4,3.1,35.4,27.8
1191,Michael Jordan,CHI,1987-88,13.0,0.1,0.132,0.7,8.8,1.0,0.603,5.9,5.5,3.2,1.6,3.1,35.0,35.0
7545,James Harden,HOU,2019-20,9.9,4.4,0.355,1.0,10.2,1.0,0.624,7.5,6.6,1.8,0.9,4.5,34.3,32.6
1535,Michael Jordan,CHI,1989-90,12.6,1.1,0.376,1.0,7.2,1.0,0.606,6.3,6.9,2.8,0.7,3.0,33.6,34.6
4487,Allen Iverson,PHI,2005-06,11.3,1.0,0.323,0.7,9.4,1.0,0.543,7.4,3.2,1.9,0.1,3.4,33.0,26.1
688,Bernard King,NYK,1984-85,12.6,0.0,0.100,0.6,7.7,1.0,0.585,3.7,5.8,1.3,0.3,3.7,32.9,26.8
2111,Michael Jordan,CHI,1992-93,12.7,1.0,0.352,0.9,6.1,1.0,0.564,5.5,6.7,2.8,0.8,2.7,32.6,31.6
1355,Michael Jordan,CHI,1988-89,11.9,0.3,0.276,0.8,8.3,1.0,0.613,8.0,8.0,2.9,0.8,3.6,32.5,37.0


### Get summary of adjusted ppg leaders comparing to actual ppg

In [138]:
differential = pd.DataFrame(columns= ['PLAYER', 'SEASON','EFF', 'TS_PCT', 'OldPTS','NewPTS', 'PTSDiff'])
differential['PLAYER'] = all_adjusted.sort_values(by=['PLAYER','SEASON']).reset_index(drop=True)['PLAYER']
differential['SEASON'] = all_adjusted.sort_values(by=['PLAYER','SEASON']).reset_index(drop=True)['SEASON']
differential['TS_PCT'] = all_adjusted.sort_values(by=['PLAYER','SEASON']).reset_index(drop=True)['TS_PCT']
differential['EFF']= all_adjusted.sort_values(by=['PLAYER','SEASON']).reset_index(drop=True)['EFF'].round(1)
differential['NewPTS'] = all_adjusted.sort_values(by=['PLAYER','SEASON']).reset_index(drop=True)['PTS'].round(1)
differential['OldPTS'] = all_data.sort_values(by=['PLAYER','SEASON']).reset_index(drop=True)['PTS']
differential['PTSDiff'] = differential['NewPTS'] - differential['OldPTS']

In [139]:
differential.sort_values(by=['NewPTS'], ascending=False).head(25)

,PLAYER,SEASON,EFF,TS_PCT,OldPTS,NewPTS,PTSDiff
5252,Michael Jordan,1986-87,31.9,0.573,37.1,37.9,0.8
3239,James Harden,2018-19,33.1,0.617,36.1,36.2,0.1
4371,Kobe Bryant,2005-06,27.8,0.568,35.4,36.0,0.6
5255,Michael Jordan,1989-90,34.6,0.641,33.6,35.7,2.1
5253,Michael Jordan,1987-88,35.0,0.611,35.0,35.5,0.5
5258,Michael Jordan,1992-93,31.6,0.590,32.6,34.8,2.2
3240,James Harden,2019-20,32.6,0.626,34.3,34.4,0.1
2646,George Gervin,1981-82,24.8,0.587,32.3,34.0,1.7
5254,Michael Jordan,1988-89,37.0,0.633,32.5,33.6,1.1
185,Allen Iverson,2005-06,26.1,0.549,33.0,33.3,0.3


In [162]:
top500 = differential.sort_values(by=['NewPTS'], ascending=False).head(500)

In [163]:
top500

,PLAYER,SEASON,EFF,TS_PCT,OldPTS,NewPTS,PTSDiff
5252,Michael Jordan,1986-87,31.9,0.573,37.1,37.9,0.8
3239,James Harden,2018-19,33.1,0.617,36.1,36.2,0.1
4371,Kobe Bryant,2005-06,27.8,0.568,35.4,36.0,0.6
5255,Michael Jordan,1989-90,34.6,0.641,33.6,35.7,2.1
5253,Michael Jordan,1987-88,35.0,0.611,35.0,35.5,0.5
...,...,...,...,...,...,...,...
7155,Tim Duncan,1998-99,25.3,0.554,21.7,23.0,1.3
6879,Stephon Marbury,2002-03,20.2,0.532,22.3,23.0,0.7
3582,Jerry Stackhouse,2001-02,17.3,0.542,21.4,23.0,1.6
4767,Luka Doncic,2018-19,21.6,0.565,21.2,23.0,1.8


In [141]:
import plotly.graph_objects as go

In [164]:
fig = go.Figure(data=go.Scatter(x=top500['NewPTS'],
                                y=top500['EFF'],
                                mode='markers',
                                marker=dict(size=(top500['EFF']),
                                color=top500['PTSDiff'],
                                colorscale=[[0, 'blue'], [1, 'red']]),
                                text=top500[['PLAYER', 'SEASON']],
                                hovertemplate=
                                "<b>%{text}</b><br>"+
                                "<b>PPG</b>: %{x}<br>"+
                                "<b>EFF</b>: %{y}<br>"+
                                "<b>PTS Adj</b>: <b>%{marker.color:.2f}</b><br>"
                                )) # hover text goes here

fig.update_layout(title='EFF vs Adjusted PPG', xaxis_title='Adjusted PPG',
                   yaxis_title='EFF',
                                template='plotly_dark' )
fig.show()

In [165]:
fig.write_html("C:/Users/jreim/Documents/Coding/AnacondaProjects/Basketball//EFFvsPPGchart.html")